In [2]:
import requests
import pandas as pd
import duckdb
import os

# Make sure the 'data' folder exists
os.makedirs('../data', exist_ok=True)

# Connect (or create) DuckDB database
con = duckdb.connect(database='../data/football.duckdb')

In [4]:
API_TOKEN = "dce8c5264b254babaf1ffb275554d858"
headers = {"X-Auth-Token": API_TOKEN}

matches_url = "https://api.football-data.org/v4/competitions/PL/matches?season=2025"
response = requests.get(matches_url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Matches fetched successfully!")
else:
    print("Error fetching matches:", response.status_code)
    data = {}

# Flatten JSON to DataFrame
matches = data.get('matches', [])
df_matches = pd.json_normalize(matches)

# Preview
df_matches.head()

Matches fetched successfully!


,id,utcDate,status,matchday,stage,group,lastUpdated,referees,area.id,area.name,...,awayTeam.shortName,awayTeam.tla,awayTeam.crest,score.winner,score.duration,score.fullTime.home,score.fullTime.away,score.halfTime.home,score.halfTime.away,odds.msg
0,537785,2025-08-15T19:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-10-12T00:21:08Z,"[{'id': 11580, 'name': 'Anthony Taylor', 'type...",2072,England,...,Bournemouth,BOU,https://crests.football-data.org/bournemouth.png,HOME_TEAM,REGULAR,4.0,2.0,1.0,0.0,Activate Odds-Package in User-Panel to retriev...
1,537786,2025-08-16T11:30:00Z,FINISHED,1,REGULAR_SEASON,None,2025-10-12T00:21:08Z,"[{'id': 11585, 'name': 'Craig Pawson', 'type':...",2072,England,...,Newcastle,NEW,https://crests.football-data.org/67.png,DRAW,REGULAR,0.0,0.0,0.0,0.0,Activate Odds-Package in User-Panel to retriev...
2,537787,2025-08-16T14:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-10-12T00:21:08Z,"[{'id': 213813, 'name': 'Sam Barrott', 'type':...",2072,England,...,Fulham,FUL,https://crests.football-data.org/63.png,DRAW,REGULAR,1.0,1.0,0.0,0.0,Activate Odds-Package in User-Panel to retriev...
3,537789,2025-08-16T14:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-10-12T00:21:08Z,"[{'id': 11446, 'name': 'Robert Jones', 'type':...",2072,England,...,West Ham,WHU,https://crests.football-data.org/563.png,HOME_TEAM,REGULAR,3.0,0.0,0.0,0.0,Activate Odds-Package in User-Panel to retriev...
4,537790,2025-08-16T14:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-10-12T00:21:08Z,"[{'id': 11605, 'name': 'Michael Oliver', 'type...",2072,England,...,Burnley,BUR,https://crests.football-data.org/328.png,HOME_TEAM,REGULAR,3.0,0.0,1.0,0.0,Activate Odds-Package in User-Panel to retriev...


In [5]:
con.execute("CREATE or REPLACE TABLE IF NOT EXISTS matches AS SELECT * FROM df_matches")
print("Matches saved to DuckDB.")

Matches saved to DuckDB.


In [6]:
teams_url = "https://api.football-data.org/v4/competitions/PL/teams"

response = requests.get(teams_url, headers=headers)
if response.status_code == 200:
    teams_data = response.json()
    print("Teams fetched successfully!")
else:
    print("Error fetching teams:", response.status_code)
    teams_data = {}

df_teams = pd.json_normalize(teams_data.get('teams', []))
df_teams.head()

# Save to DuckDB
con.execute("CREATE OR REPLACED TABLE teams AS SELECT * FROM df_teams")
print("Teams saved to DuckDB.")

Teams fetched successfully!
Teams saved to DuckDB.


In [22]:
import time

# Prepare storage for all players
all_players = []
failed_teams = []

# Rate limit parameters
REQUESTS_PER_MIN = 10
SLEEP_BETWEEN_CALLS = 60 / REQUESTS_PER_MIN  # seconds

for i, team_id in enumerate(df_teams['id'], start=1):
    success = False
    for attempt in range(3):  # Try up to 3 times per team
        resp = requests.get(f"https://api.football-data.org/v4/teams/{team_id}", headers=headers)
        if resp.status_code == 200:
            team_data = resp.json()
            players = team_data.get('squad', [])
            for p in players:
                p['team_id'] = team_id
                all_players.append(p)
            success = True
            break
        else:
            time.sleep(2)  # short retry delay if failed

    if not success:
        failed_teams.append(team_id)

    # Rate limit: pause before the next request
    time.sleep(SLEEP_BETWEEN_CALLS)

    # Optional progress feedback
    print(f"Processed {i}/{len(df_teams)} teams")

# Convert to DataFrame
df_players = pd.json_normalize(all_players)

# Remove duplicate players
df_players = df_players.drop_duplicates(subset=['id'])

# Show missing teams this run
missing_team_names = df_teams[df_teams['id'].isin(failed_teams)][['id', 'name']]
print("\nTeams missing players this run:")
print(missing_team_names)

# Reload DuckDB table safely
con.execute("CREATE OR REPLACE TABLE players AS SELECT * FROM df_players")
print("\nPlayers table updated in DuckDB.")

Processed 1/20 teams
Processed 2/20 teams
Processed 3/20 teams
Processed 4/20 teams
Processed 5/20 teams
Processed 6/20 teams
Processed 7/20 teams
Processed 8/20 teams
Processed 9/20 teams
Processed 10/20 teams
Processed 11/20 teams
Processed 12/20 teams
Processed 13/20 teams
Processed 14/20 teams
Processed 15/20 teams
Processed 16/20 teams
Processed 17/20 teams
Processed 18/20 teams
Processed 19/20 teams
Processed 20/20 teams

Teams missing players this run:
Empty DataFrame
Columns: [id, name]
Index: []

Players table updated in DuckDB.


In [18]:
#print("Sample matches:")
#print(con.execute("SELECT * FROM matches LIMIT 5").fetchdf())

#print("Sample teams:")
#print(con.execute("SELECT * FROM teams LIMIT 5").fetchdf())

#print("Sample players:")
#print(con.execute("SELECT * FROM players LIMIT 5").fetchdf())


In [9]:
df_players

,id,name,position,dateOfBirth,nationality,team_id
0,3189,Kepa Arrizabalaga,Goalkeeper,1994-10-03,Spain,57
1,4832,David Raya,Goalkeeper,1995-09-15,Spain,57
2,171455,Tommy Setford,Goalkeeper,2006-03-13,England,57
3,285128,Max Dowman,Midfield,2009-12-31,England,57
4,285598,Andre Harriman-Annous,Offence,2007-12-27,England,57
...,...,...,...,...,...,...
298,178644,Eliezer Mayenda,Centre-Forward,2005-05-08,Spain,71
299,172275,Romaine Mundle,Left Winger,2003-04-24,England,71
300,172127,Dan Neil,Central Midfield,2001-11-30,England,71
301,172123,Dennis Cirkin,Left-Back,2002-04-06,England,71


In [21]:
# Set of all team IDs from the teams table
team_ids_in_teams = set(df_teams['id'])

# Set of team IDs that appear in your players table
team_ids_in_players = set(df_players['team_id'])

# Teams with missing players
missing_teams = team_ids_in_teams - team_ids_in_players

# Show missing teams with names
missing_team_names = df_teams[df_teams['id'].isin(missing_teams)][['id', 'name']]
print("Teams missing players:")
print(missing_team_names)


Teams missing players:
     id                        name
11   76  Wolverhampton Wanderers FC
12  328                  Burnley FC


In [23]:
# Set of all team IDs from the teams table
team_ids_in_teams = set(df_teams['id'])

# Set of team IDs that appear in your players table
team_ids_in_players = set(df_players['team_id'])

# Teams with missing players
missing_teams = team_ids_in_teams - team_ids_in_players

# Show missing teams with names
missing_team_names = df_teams[df_teams['id'].isin(missing_teams)][['id', 'name']]
print("Teams missing players:")
print(missing_team_names)

Teams missing players:
Empty DataFrame
Columns: [id, name]
Index: []


In [30]:
"""
Full ETL script for Premier League Football Data
------------------------------------------------
Fetches teams, matches, and players from Football-Data.org API,
loads them into DuckDB or Snowflake depending on configuration.

Features:
- Logging to both console and timestamped file
- Rate limiting and retry logic for API calls
- Config-driven database connections
- Docstrings and inline comments for clarity
"""

import os
import time
import logging
from datetime import datetime
import requests
import pandas as pd
import yaml

# -------------------------
# Logging setup
# -------------------------
os.makedirs("../logs", exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
log_filename = f"../logs/football_etl_log_{timestamp}.log"

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# File handler (new file each run)
file_handler = logging.FileHandler(log_filename, mode="w")
file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

# Console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(file_formatter)
logger.addHandler(console_handler)

logger.propagate = False  # prevent double output in Jupyter
logging.info(f"Logging initialized. Logs will go to {log_filename} and console.")

# -------------------------
# Load configuration
# -------------------------
def load_config(config_path="../config/config.yaml"):
    """Load YAML configuration file."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config

# -------------------------
# Database connection
# -------------------------
def get_db_connection(config):
    """Create a database connection based on config."""
    db_type = config['database']['type'].lower()

    if db_type == 'duckdb':
        import duckdb
        os.makedirs(os.path.dirname(config['database']['duckdb_path']), exist_ok=True)
        con = duckdb.connect(database=config['database']['duckdb_path'])
        logging.info(f"Connected to DuckDB at {config['database']['duckdb_path']}")
        return con

    elif db_type == 'snowflake':
        import snowflake.connector
        con = snowflake.connector.connect(
            user=config['database']['snowflake']['user'],
            password=config['database']['snowflake']['password'],
            account=config['database']['snowflake']['account'],
            database=config['database']['snowflake']['database'],
            warehouse=config['database']['snowflake']['warehouse']
        )
        logging.info("Connected to Snowflake")
        return con

    else:
        raise ValueError(f"Unsupported database type: {db_type}")

# -------------------------
# API Fetch Functions
# -------------------------
def fetch_json(url, headers, retries=3, sleep_between=2):
    """Fetch JSON data from a URL with retry logic."""
    for attempt in range(retries):
        try:
            resp = requests.get(url, headers=headers)
            if resp.status_code == 200:
                return resp.json()
            else:
                logging.warning(f"Attempt {attempt+1} failed: {resp.status_code} for {url}")
        except requests.RequestException as e:
            logging.error(f"Request exception: {e}")
        time.sleep(sleep_between)
    logging.error(f"Failed to fetch: {url}")
    return None

def fetch_teams(headers, con, db_type):
    """Fetch Premier League teams and store in database."""
    url = "https://api.football-data.org/v4/competitions/PL/teams"
    data = fetch_json(url, headers)
    if data and "teams" in data:
        df_teams = pd.json_normalize(data['teams'])
        if db_type == "duckdb":
            con.execute("CREATE OR REPLACE TABLE teams AS SELECT * FROM df_teams")
        elif db_type == "snowflake":
            con.cursor().execute("CREATE OR REPLACE TABLE teams AS SELECT * FROM df_teams")
        logging.info("Teams table updated in DB.")
        return df_teams
    else:
        logging.error("Teams data not available.")
        return pd.DataFrame()

def fetch_matches(headers, con, db_type, season=2025):
    """Fetch Premier League matches for a season and store in database."""
    url = f"https://api.football-data.org/v4/competitions/PL/matches?season={season}"
    data = fetch_json(url, headers)
    if data and "matches" in data:
        df_matches = pd.json_normalize(data['matches'])
        if db_type == "duckdb":
            con.execute("CREATE OR REPLACE TABLE matches AS SELECT * FROM df_matches")
        elif db_type == "snowflake":
            con.cursor().execute("CREATE OR REPLACE TABLE matches AS SELECT * FROM df_matches")
        logging.info("Matches table updated in DB.")
        return df_matches
    else:
        logging.error("Matches data not available.")
        return pd.DataFrame()

def fetch_team_players(team_id, headers, retries=3, sleep_between=2):
    """Fetch players for a single team."""
    for attempt in range(retries):
        try:
            url = f"https://api.football-data.org/v4/teams/{team_id}"
            resp = requests.get(url, headers=headers)
            if resp.status_code == 200:
                squad = resp.json().get('squad', [])
                for player in squad:
                    player['team_id'] = team_id
                return squad
            else:
                logging.warning(f"Attempt {attempt+1} failed for team {team_id}: {resp.status_code}")
        except requests.RequestException as e:
            logging.error(f"Request exception for team {team_id}: {e}")
        time.sleep(sleep_between)
    logging.warning(f"All attempts failed for team {team_id}")
    return []

def fetch_all_players(df_teams, headers, con, db_type, requests_per_min=10, retries=3, sleep_between=2):
    """Fetch players for all teams with rate limiting and store in database."""
    all_players = []
    failed_teams = []
    sleep_between_calls = 60 / requests_per_min

    for i, team_id in enumerate(df_teams['id'], start=1):
        players = fetch_team_players(team_id, headers, retries, sleep_between)
        if players:
            all_players.extend(players)
        else:
            failed_teams.append(team_id)
        logging.info(f"Processed {i}/{len(df_teams)} teams")
        time.sleep(sleep_between_calls)

    df_players = pd.json_normalize(all_players).drop_duplicates(subset=['id'])

    if db_type == "duckdb":
        con.execute("CREATE OR REPLACE TABLE players AS SELECT * FROM df_players")
    elif db_type == "snowflake":
        con.cursor().execute("CREATE OR REPLACE TABLE players AS SELECT * FROM df_players")

    logging.info("Players table updated in DB.")

    if failed_teams:
        missing_team_names = df_teams[df_teams['id'].isin(failed_teams)][['id','name']]
        logging.warning("Teams missing players this run:")
        logging.warning("\n%s", missing_team_names)

    return df_players

# -------------------------
# Main ETL orchestrator
# -------------------------
def main(config_dict=None):
    """Main ETL function."""
    if config_dict is None:
        config = load_config()
    else:
        config = config_dict

    API_TOKEN = config['api']['token']
    HEADERS = {"X-Auth-Token": API_TOKEN}
    REQUESTS_PER_MIN = config['etl']['requests_per_min']
    RETRIES_PER_TEAM = config['etl']['retries_per_team']
    SLEEP_BETWEEN_RETRIES = config['etl']['sleep_between_retries']

    con = get_db_connection(config)
    db_type = config['database']['type'].lower()

    df_teams = fetch_teams(HEADERS, con, db_type)
    if df_teams.empty:
        logging.error("No teams fetched. Aborting ETL.")
        return
    df_matches = fetch_matches(HEADERS, con, db_type)
    df_players = fetch_all_players(df_teams, HEADERS, con, db_type,
                                   requests_per_min=REQUESTS_PER_MIN,
                                   retries=RETRIES_PER_TEAM,
                                   sleep_between=SLEEP_BETWEEN_RETRIES)

    logging.info("ETL run complete.")

# -------------------------
# CLI run
# -------------------------
if __name__ == "__main__":
    main()


2025-10-13 22:02:48,285 - INFO - Logging initialized. Logs will go to ../logs/football_etl_log_20251013220248.log and console.
2025-10-13 22:02:48,285 - INFO - Logging initialized. Logs will go to ../logs/football_etl_log_20251013220248.log and console.
2025-10-13 22:02:48,285 - INFO - Logging initialized. Logs will go to ../logs/football_etl_log_20251013220248.log and console.
2025-10-13 22:02:48,285 - INFO - Logging initialized. Logs will go to ../logs/football_etl_log_20251013220248.log and console.
2025-10-13 22:02:48,285 - INFO - Logging initialized. Logs will go to ../logs/football_etl_log_20251013220248.log and console.
2025-10-13 22:02:48,285 - INFO - Logging initialized. Logs will go to ../logs/football_etl_log_20251013220248.log and console.
2025-10-13 22:02:48,331 - INFO - Connected to DuckDB at ../data/football.duckdb
2025-10-13 22:02:48,331 - INFO - Connected to DuckDB at ../data/football.duckdb
2025-10-13 22:02:48,331 - INFO - Connected to DuckDB at ../data/football.duckd